In [ ]:
import pandas as pd
import time
#from config import headers,url,Cookie,base_url,weiboComment,excel_name,txt_name
import json
from time import sleep
#import pymysql
import random
import numpy as np
import datetime
from tqdm import tqdm
from datetime import datetime, timedelta
import re
import sys

from scipy.stats.stats import pearsonr
from scipy import stats
from scipy.stats.stats import pearsonr
from scipy import stats
import scipy
import matplotlib.pyplot as plt
import pickle
import plotly.graph_objects as go
import seaborn as sns

In [ ]:
with open("input file path\map_com2.pkl", 'rb') as handle:
    map_com2 = pickle.load(handle)
    
result_df = pd.read_pickle("input file path\result_df.pkl")  
result_df


In [ ]:
I=30
node_label=list()
sankey_dict=dict()
sankey_dict2=dict()
color_index=1
ind=0
x=list()
y=list()
colors=['lightgrey','antiquewhite','crimson', 'cyan', 'aquamarine', 'azure', 'cadetblue',
            'chartreuse', 'chocolate', 'coral', 'cornflowerblue',
            'cornsilk',  'darkcyan',
            'darkgoldenrod', 'darkgray', 'darkgrey', 'darkgreen',
            'darkkhaki', 'darkmagenta', 'darkolivegreen', 'darkorange','aliceblue','darkblue', 'bisque', 'black','crimson','beige', 'cyan', 'blue',
            'blueviolet', 'brown', 'antiquewhite', 'blanchedalmond']
colors+=colors
colors+=colors
link_source=list()
link_target=list()
link_value=list()
node_profile=pd.DataFrame(columns=['step','x','y','color','size','date','com_code'])

for i in range(1,I):
    #print(i,'--------------------')
    sankey_node_w=dict()
    d1=list(result_df.index)[i-1]
    d2=list(result_df.index)[i]
    sankey_dict[d2]=dict()
    sankey_dict2[d2]=dict()
    
        
    nodes_old_to=list(set(list(np.asarray(result_df.loc[d2,0])[:,1])))
    nodes_old_to
    nodes_size=dict()
    for n in nodes_old_to:
        nodes_size[n]=sum([t[2] for t in result_df.loc[d2,0] if t[1]==n])
    nodes_size=dict(sorted(nodes_size.items(), key=lambda item: item[1],reverse=True))
    thresh=0.01
    if i==1:
        key_nodes0=list()
        y_ind=0.1

        key_nodes_temp=[list(nodes_size.keys())[i] for i in range(len(nodes_size.values())) if list(nodes_size.values())[i]/sum(list(nodes_size.values()))>=0.01]
        sankey_dict[d1]=dict()
        sankey_dict[d2]=dict()
        sankey_dict2[d1]=dict()
        sankey_dict2[d2]=dict()
        
        
        #add initial nodes
        for t in result_df.loc[d2,0]:
            if t[1] in key_nodes_temp:
                if t[0]!=0 and t[1]<10000 and t[0] not in key_nodes0 and t[2]>=np.sum(np.asarray(result_df.loc[d2,0])[:,2])*0.05:
                    node_label.append(ind)
                    key_nodes0.append(t[0])
                    sankey_dict[d1][t[0]]=ind
                    sankey_dict2[d1][ind]=t[0]
                    node_profile.loc[ind,:]=[i-1,i,y_ind,color_index,t[2],d1,map_com2[d1][t[0]]]
                    ind+=1
                    y_ind=y_ind+0.05
                    color_index=color_index+1
        
    y_ind=0.1
    key_nodes_temp=[list(nodes_size.keys())[i] for i in range(len(nodes_size.values())) if list(nodes_size.values())[i]/sum(list(nodes_size.values()))>=thresh]
    key_nodes=list()
    for k in key_nodes0+key_nodes_temp:
        if k in key_nodes0 and k in key_nodes_temp:
            key_nodes.append(k)
        elif k not in key_nodes_temp and k<10000:
            #print(k,[[ii[0],ii[1],ii[2],np.round(ii[2]/sum(list(nodes_size.values())),2)] for ii in result_df.loc[d2,0] if ii[0]==k])
            if len([ii for ii in result_df.loc[d2,0] if ii[0]==k and ii[1]<10000 and ii[2]/sum(list(nodes_size.values()))>=0.01])>0:
                key_nodes.append([ii[1] for ii in result_df.loc[d2,0] if ii[0]==k and ii[1]<10000][0])
        
    key_nodes=list(dict.fromkeys(key_nodes))
    #print(key_nodes)
    for k in key_nodes:
        
        sankey_dict[d2][k]=ind
        sankey_dict2[d2][ind]=k
        node_label.append(ind)
        node_profile.loc[ind,:]=[i,i+1,y_ind,0,sum([ii[2] for ii in result_df.loc[d2,0] if ii[1]==k]),d2,map_com2[d2][k]]
        y_ind=y_ind+0.1
        ind+=1
    
    link_lst=list()
    for n in key_nodes0:
        link_lst+=[ii for ii in result_df.loc[d2,0] if ii[0]==n and ii[1] in key_nodes]
    
    
    link_source+=[sankey_dict[d1][ii[0]] for ii in link_lst]
    link_target+=[sankey_dict[d2][ii[1]] for ii in link_lst]
    link_value+=[ii[2] for ii in link_lst]
    
    # Adjust color of nodes
    done_lst=list()
    for l in link_lst:
        if sankey_dict[d2][l[1]] not in done_lst:
            node_profile.loc[sankey_dict[d2][l[1]],'color']=node_profile.loc[sankey_dict[d1][l[0]],'color']
            done_lst.append(sankey_dict[d2][l[1]])
            
    for k in key_nodes_temp:
        #print(k)
        if k not in key_nodes0 and k not in key_nodes:
            temp=[ii for ii in result_df.loc[d2,0] if ii[0] in key_nodes0+[k] and ii[1]==k]
            
            if len(temp)>0:
                for t in temp:
                    
                    if t[0] in key_nodes0:
                        key_nodes.append(t[1])
                        sankey_dict[d2][t[1]]=ind
                        sankey_dict2[d2][ind]=t[1]
                        node_label.append(ind)
                        try:
                            node_profile.loc[ind,:]=[i,i+1,y_ind,color_index,sum([ii[2] for ii in result_df.loc[d2,0] if ii[1]==t[1]]),d2,map_com2[d2][t[1]]]
                        except:
                            node_profile.loc[ind,:]=[i,i+1,y_ind,color_index,sum([ii[2] for ii in result_df.loc[d2,0] if ii[1]==t[1]]),d2,0]
                        y_ind+=0.05
                        ind+=1
                        color_index+=1

                        link_source+=[sankey_dict[d1][t[0]]]
                        link_target+=[sankey_dict[d2][t[1]]]
                        link_value+=[t[2]]
                        break
                    else:
                        key_nodes0.append(t[0])
                        sankey_dict[d1][t[0]]=ind
                        sankey_dict2[d1][ind]=t[0]
                        node_label.append(ind)
                        node_profile.loc[ind,:]=[i-1,i,len(sankey_dict[d1].keys())*0.1+0.1,color_index,sum([ii[2] for ii in result_df.loc[d2,0] if ii[0]==t[0]]),d1,map_com2[d1][t[0]]]
                        ind+=1

                        key_nodes.append(t[1])
                        sankey_dict[d2][t[1]]=ind
                        sankey_dict2[d2][ind]=t[1]
                        node_label.append(ind)
                        try:
                            node_profile.loc[ind,:]=[i,i+1,y_ind,color_index,sum([ii[2] for ii in result_df.loc[d2,0] if ii[1]==t[1]]),d2,map_com2[d2][t[1]]]
                        except:
                            node_profile.loc[ind,:]=[i,i+1,y_ind,color_index,sum([ii[2] for ii in result_df.loc[d2,0] if ii[1]==t[1]]),d2,0]
                        y_ind+=0.05
                        ind+=1
                        color_index+=1

                        link_source+=[sankey_dict[d1][t[0]]]
                        link_target+=[sankey_dict[d2][t[1]]]
                        link_value+=[t[2]]

                        #print('new',k,'from:',t[0],', to:',t[1],', modifid_from:',sankey_dict[d1][t[0]],', modifid_to:',sankey_dict[d2][t[1]])

                        link_lst+=[t]
                        break
        
    
    key_nodes0=key_nodes
x=list(np.asarray(node_profile.loc[:,'x'])/max(np.asarray(node_profile.loc[:,'x'])))
y=list(node_profile.loc[:,'y'])
fig = go.Figure(data=[go.Sankey(
    arrangement='snap',
    node = dict(thickness = 10,
      #label =  list(np.arange(0,nodes[-1]+1,1))
        label=node_label,
        color=[colors[cc] for cc in node_profile.loc[:,'color']],
        x=list(x),
        y=list(y)
        ),
        # Add links

        link = dict(
          source =  link_source,
          target =  link_target,
          value =  link_value,
          #label = link_label
    )
    )])
fig.update_layout(font_size=8)
fig.show()